## 读
* read_csv
* read_excel
* read_hdf
* read_sql
* read_json
* read_msgpack (experimental)
* read_html
* read_gbq (experimental)
* read_stata
* read_sas
* read_clipboard
* read_pickle
## 写
* to_csv
* to_excel
* to_hdf
* to_sql
* to_json
* to_msgpack (experimental)
* to_html
* to_gbq (experimental)
* to_stata
* to_clipboard
* to_pickle
## 其他数据源
* 数据库
* 读取网页数据,网络文件
* 读取股票数据
* yahoo，世界银行等等

In [1]:
import numpy as np
import pandas as pd

### csv,txt

In [ ]:
eu12= pd.read_csv('./data/Eueo2012.csv')
eu12.head()

In [ ]:
columns = eu12.columns

In [ ]:
columns

In [ ]:
# 把第一列作为索引
eu12 = pd.read_csv('./data/Eueo2012.csv', index_col=0)
eu12.head()

In [ ]:
# 如果列没有名字，第一行会自动变成列
eu12_header = pd.read_csv('./data/Eueo2012_no_header.csv')
eu12_header.head()

In [ ]:
# 通过给header设置为None，可以告诉pandas没有列
eu12_header = pd.read_csv('./data/Eueo2012_no_header.csv', header=None)
eu12_header.head()

In [ ]:
# 如果知道了列的名字的话，可以直接进行赋值
eu12_header = pd.read_csv('./data/Eueo2012_no_header.csv', names=columns)
eu12_header.head()

In [ ]:
# 在读入数据的时候，可以告诉每一列的数据 是什么类型的数据(！！！通过字典的方式进行设定！！！)
eu12_header = pd.read_csv('./data/Eueo2012_no_header.csv', names=columns, dtype={'Goals':np.float64})
eu12_header.head()

In [ ]:
# 获取各个列的数据的类型
eu12_header.dtypes

In [ ]:
# 如果只想读某几行的话，要用到!!!usecols!!!字段，值为一个列表
eu12 = pd.read_csv('./data/Eueo2012.csv', index_col=0, usecols=['Team', 'Goals', 'Shots on target'])
eu12.head()

In [ ]:
# 如果index_label的值赋值为False，那么生成的cvs中，Team就没有了！！！！
eu12.to_csv('./data/Eueo2012_save.csv', index_label='Team')

In [ ]:
!cat ./data/Eueo2012_save.csv      # 看来cat是查看的命令

In [ ]:
!type ./data/Eueo2012_save.csv    # type也是查看的命令【这个是Windows中的命令】

In [ ]:
eu12.to_csv?

### 进阶

In [ ]:
!cat ./data/Eueo2012_del.txt

In [ ]:
# 因为分割是用的“？”，所以会分不出来
eu12=pd.read_table('./data/Eueo2012_del.txt', index_col=0)
eu12.head()

In [ ]:
# 我们可以根据需要设置sep的值，来灵活处理，这个参数csv中也适用。
eu12=pd.read_table('./data/Eueo2012_del.txt', sep='?', index_col=0)
eu12.head()

In [ ]:
!cat ./data/Eueo2012_skip.txt

In [ ]:
# 而且我们可以设置哪些行是要跳过的,直接传值是要跳过多少行
# ！！！！有个地方是没想到的：读txt文件也可以用读csv的方法来读！！！
eu12=pd.read_csv('./data/Eueo2012_skip.txt', sep='?', index_col=0, skiprows=[0,1])
eu12

In [ ]:
# 直接在skiprows传-1是不起作用的，这个时候就需要用到skipfooter这个参数了。。！！！！参数的值是对应要跳过的行数！！！！
# 而且最好通过engine来指定引擎是Python，要不会有警告的！！！
eu12=pd.read_csv('./data/Eueo2012_skip.txt', sep='?', index_col=0, skiprows=[0,1], skipfooter=1, engine='python')
eu12

In [ ]:
#--！！也可以通过nrow来指定读入多少行数据！！--#
eu12 = pd.read_table('./data/Eueo2012_del.txt', sep='?', index_col=0, nrows=5)
eu12

In [ ]:
eu12 = pd.read_csv('./data/Eueo2012_del.txt', header=0, skiprows=5, nrows=5, sep='?', index_col=0, names=['Goals', 'Shot on target'])
eu12

### excel

In [ ]:
# 读取excel的时候，需要特别指定sheet_name，来告诉读取那个sheet。
eu12 = pd.read_excel('./data/Eueo2012_excel.xlsx', sheet_name="Eueo2011")
eu12.head()

In [ ]:
# 通过设置index为False的话，就不会把DataFrame中的索引页添加上去了！
eu12.to_excel('./data/test.xlsx', index=False)

In [ ]:
# 新建了表之后，还可以指定表的name
eu12.to_excel('./data/test1.xlsx', sheet_name='eu12',index=False)

In [ ]:
eu12.to_excel?

一个jupyter notebook的技巧:`esc` + `a`在格子的上方插入格子；`esc` + `b`在格子的下方插入格子；`esc` + `d`(两次)可以删除当前的格子

In [ ]:
# 把多个数据写入同一个excel文件
from pandas import ExcelWriter

In [ ]:
with ExcelWriter('./data/test3.xlsx') as writer:
    eu12.to_excel(writer, sheet_name='eu12', index=False)
    eu12.to_excel(writer, sheet_name='eu11', index=False)

### json

In [ ]:
!cat ./data/eueo2012.json

In [ ]:
eu12_json = pd.read_json('./data/eueo2012.json')    # 读进来的顺序变了
eu12_json.head()

## 数据库

In [ ]:
import sqlite3
# 写入
connection = sqlite3.connect('./data/eueo2012.sqlite')
eu12.to_sql('EU12_DATA', connection, if_exists='replace')
# 提交，最后关闭
connection.commit()
connection.close()

In [ ]:
# 读取
connection = sqlite3.connect('./data/eueo2012.sqlite')
eu12_sql = pd.io.sql.read_sql('SELECT * FROM EU12_DATA;', connection, index_col='index')
# 关闭连接
connection.close()
eu12_sql.head()

In [ ]:
# 进行条件查询
connection = sqlite3.connect('./data/eueo2012.sqlite')
query = 'SELECT * from EU12_DATA where Goals>5'
items = pd.io.sql.read_sql(query, connection, index_col='index')
connection.close()
items

### hdf5

In [ ]:
df = pd.DataFrame(np.random.randn(10, 3), index=pd.date_range('11/11/2016',periods=10), columns=['A', 'B', 'C'])
df

In [ ]:
# 存储是数据到hdf5
hdf = pd.HDFStore('./data/hdf.h5')

In [ ]:
hdf['df'] = df

In [ ]:
hdf

In [ ]:
# 从文件中读取
read_hdf = pd.HDFStore('./data/hdf.h5')
df = read_hdf['df']
df

In [ ]:
df.ix[0].A = 1
df

In [ ]:
hdf['df'] = df

In [ ]:
pd.HDFStore('./data/hdf.h5')['df'].head()

## 读取web数据

In [2]:
url = "http://worldcup.2014.163.com/playerrank/index.html"

In [3]:
eu2012=pd.read_html(url)

In [4]:
eu2012

[   Unnamed: 0  排名     球员  数据    球队  Unnamed: 5
 0         NaN   1  罗德里格斯   6  哥伦比亚         NaN
 1         NaN   2     穆勒   5    德国         NaN
 2         NaN   3    内马尔   4    巴西         NaN
 3         NaN   4     梅西   4   阿根廷         NaN
 4         NaN   5    范佩西   4    荷兰         NaN,
    Unnamed: 0  排名     球员  数据    球队  Unnamed: 5
 0         NaN   1    内马尔   1    巴西         NaN
 1         NaN   2   萨马拉斯   1    希腊         NaN
 2         NaN   3    范佩西   1    荷兰         NaN
 3         NaN   4   耶迪纳克   1  澳大利亚         NaN
 4         NaN   5  罗德里格斯   1  哥伦比亚         NaN,
    Unnamed: 0  排名     球员  数据    球队  Unnamed: 5
 0         NaN   1     穆勒   4    德国         NaN
 1         NaN   2    克罗斯   4    德国         NaN
 2         NaN   3   夸德拉多   4  哥伦比亚         NaN
 3         NaN   4    布林德   3    荷兰         NaN
 4         NaN   5  罗德里格斯   2  哥伦比亚         NaN,
    Unnamed: 0  排名     球员  数据    球队  Unnamed: 5
 0         NaN   1    本泽马  26    法国         NaN
 1         NaN   2   迪马利亚  21   阿根廷  

In [5]:
eu2012[0]

,Unnamed: 0,排名,球员,数据,球队,Unnamed: 5
0,NaN,1,罗德里格斯,6,哥伦比亚,NaN
1,NaN,2,穆勒,5,德国,NaN
2,NaN,3,内马尔,4,巴西,NaN
3,NaN,4,梅西,4,阿根廷,NaN
4,NaN,5,范佩西,4,荷兰,NaN


### remote data

In [6]:
# import pandas.io.data as web
# 非常好用，不需要知道连接！！！
import pandas_datareader.data as web
import datetime
start= datetime.datetime(2016, 11, 1)
end = datetime.datetime(2016, 11, 11)
apple = web.DataReader('AAPL', 'yahoo', start, end)
apple.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-10-31,113.650002,114.230003,113.199997,113.540001,111.172279,26419400
2016-11-01,113.459999,113.769997,110.529999,111.489998,109.165031,43825800
2016-11-02,111.400002,112.349998,111.230003,111.589996,109.262947,28331700
2016-11-03,110.980003,111.459999,109.550003,109.830002,108.091774,26932600
2016-11-04,108.529999,110.250000,108.110001,108.839996,107.117447,30837000


In [ ]:
# tushare是个很不错的财经数据的读取包